<img src="./Assets/1.1 AgWorkHeaderImage.png">

## Multi Agent Workflows

<img src="./Assets/4.1 Multi Agent.png">

---

#### Load All Keys

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

## Recall: AstraDB Vector Tool (Custom CrewAI Tools)

In [2]:
Token=os.environ["ASTRA_TOKEN"]
EndPoint=os.environ["ASTRA_ENDPOINT"]

In [3]:
from astrapy import DataAPIClient
from astrapy.data_types import DataAPIVector
from openai import OpenAI

def search_documents(query: str, top_k: int = 5) -> list[str]:
    # Set up OpenAI
    openai_client = OpenAI()
    query_embedding = openai_client.embeddings.create(
        model="text-embedding-3-small",
        input=query,
        encoding_format="float"
    ).data[0].embedding

    # Connect to Astra
    astra_client = DataAPIClient(Token)
    db = astra_client.get_database_by_api_endpoint(EndPoint)
    collection = db.get_collection("CollectionSmallEmbeddings")

    # Query Astra
    cursor = collection.find(
        sort={"$vector": DataAPIVector(query_embedding)},
        limit=top_k,
        include_similarity=True
    )

    # Return documents
    results = []
    for doc in cursor:
        text = doc.get("text") or str(doc)
        score = doc.get("$similarity", 0)
        results.append(f"[{score:.4f}] {text}")
    return results


In [4]:
from crewai.tools import BaseTool

class AstraSearchTool(BaseTool):
    name: str = "Vector Tool"
    description: str = "Search the vector store for information about employee leave policy of AKAIWorks"

    def _run(self, query: str) -> str:
        # This is where your vector search logic goes
        results = search_documents(query)
        return "\n".join(results)

2025-05-16 12:40:32,796 - 6282653696 - telemetry.py-telemetry:50 - ERROR: HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Read timed out. (read timeout=30)


In [5]:
astra_tool = AstraSearchTool()

---

In [6]:
from crewai import LLM

llm = LLM(
    model="gpt-4o-mini",
    temperature=0.8,
    max_tokens=150,
    top_p=0.9,
    frequency_penalty=0.1,
    presence_penalty=0.1,
    stop=["END"],
    seed=42
)

## Agents

In [49]:
from crewai import Agent
from crewai_tools import SerperDevTool

In [50]:
task_planner = Agent(
    role="Task Planning Analyst",
    goal="Break down user questions into subtasks.",
    backstory="You understand complex goals and decompose them into logical steps.",
    llm=llm, verbose=True
)

In [51]:
query_refiner = Agent(
    role="Query Refinement Specialist",
    goal="Improve vague or broad queries for better retrieval.",
    backstory="You turn fuzzy queries into laser-focused search prompts.",
    llm=llm, verbose=True
)


In [52]:
tool_selector = Agent(
    role="Retrieval Strategy Advisor",
    goal="Choose which data source (Web, Vector, LLM) is best.",
    backstory="You understand which retrieval method fits which query.",
    llm=llm, verbose=True
)

In [ ]:
iterative_retriever = Agent(
    role="Iterative Retrieval Specialist",
    goal="Retrieve complete, high-quality information through iterative reflection and refinement.",
    backstory=(
        "You're a methodical research expert. Instead of stopping at one search, you evaluate your "
        "initial results, refine your approach, and perform additional rounds of retrieval if needed. "
        "You're committed to ensuring no key detail is missed."
    ),
    tools=[SerperDevTool(), astra_tool], 
    llm=llm,
    verbose=True
)

In [54]:
context_builder = Agent(
    role="Context Curator",
    goal="Select top passages from retrieved results.",
    backstory="You’re an information architect focused on building accurate context windows.",
    llm=llm, verbose=True
)

In [55]:
summarizer = Agent(
    role="Content Summarizer",
    goal="Summarize large retrieved content when needed.",
    backstory="You condense dense material into clean, clear summaries.",
    llm=llm, verbose=True
)


In [56]:
generator = Agent(
    role="Final Answer Composer",
    goal="Generate a clean, helpful, source-aware answer.",
    backstory="You produce clear answers from compiled research.",
    llm=llm, verbose=True
)


In [57]:
critic = Agent(
    role="AI Response Critic",
    goal="Check final answer for factuality and clarity.",
    backstory="You verify correctness and offer revisions when needed.",
    llm=llm, verbose=True
)

## Tasks

In [58]:
from crewai import Task

task_planner_task = Task(
    description="Analyze the user question '{user_input}' and break it into subtasks.",
    expected_output=(
        "A step-by-step plan. Example:\n"
        "1. Refine the query\n2. Retrieve info\n3. Generate answer\n4. Critique result"
    ),
    agent=task_planner
)

In [59]:
query_refinement_task = Task(
    description="Refine this query: '{user_input}' so it’s optimized for search and retrieval.",
    expected_output="A clear, specific refined query.",
    agent=query_refiner
)

In [60]:
tool_strategy_task = Task(
    description="Given the refined query, choose whether to use [WEB], [VECTOR], or [LLM]. Justify your choice.",
    expected_output="List like [WEB, VECTOR] and a short rationale.",
    agent=tool_selector
)

In [61]:
iterative_retrieval_task = Task(
    description=(
        "Use the refined query to perform information retrieval. "
        "Search relevant sources — use Serper for [WEB], VectorTool for [VECTOR]. "
        "After retrieving, reflect on the completeness and quality of results. "
        "If needed, refine your query and search again. Do this iteratively (up to 3 rounds max). "
        "At the end, return the best information you've gathered from all iterations, summarized by source."
    ),
    expected_output=(
        "A structured summary:\nWEB: ...\nVECTOR: ...\nIterations used: N\n"
        "Content must be accurate and comprehensive."
    ),
    agent=iterative_retriever,
    tools=[SerperDevTool(), astra_tool]
)

In [62]:
context_builder_task = Task(
    description="From the retrieval results, pick the most relevant content to build a high-quality context.",
    expected_output="A structured, clean context window.",
    agent=context_builder
)

In [63]:
summarization_task = Task(
    description="If the context is too long, summarize it clearly without losing key facts.",
    expected_output="~200 word summary of the context content.",
    agent=summarizer
)

In [64]:
generation_task = Task(
    description="Write the final answer to '{user_input}' using the context. Mention sources used.",
    expected_output="Final answer with a [Sources used: ...] section.",
    agent=generator
)

In [65]:
critic_task = Task(
    description="Critique the answer for accuracy, clarity, and completeness. Suggest edits if needed.",
    expected_output="Evaluation: [Approved] or [Needs Revision] + possible revision.",
    agent=critic
)

## Crew

In [66]:
from crewai import Crew, Process

rag_sequential_crew = Crew(
    agents=[
        task_planner,
        query_refiner,
        tool_selector,
        iterative_retriever,
        context_builder,
        summarizer,
        generator,
        critic,
    ],
    tasks=[
        task_planner_task,
        query_refinement_task,
        tool_strategy_task,
        iterative_retrieval_task,
        context_builder_task,
        summarization_task,
        generation_task,
        critic_task,
    ],
    process=Process.sequential
)


In [67]:
user_input = input("🧠 Enter your question: ")
result = rag_sequential_crew.kickoff(inputs={"user_input": user_input})

# Agent: Task Planning Analyst
## Task: Analyze the user question 'what is the parental leave policy?' and break it into subtasks.


# Agent: Task Planning Analyst
## Final Answer: 
1. Refine the query  
   - Identify the specific context: Is the inquiry about a particular company, country, or type of employment?  
   - Clarify the user's intent: Are they looking for a general overview, specific details, or comparisons with other policies?  

2. Retrieve info  
   - Gather data from reliable sources: Check company HR policies, government websites, or labor law resources relevant to the context identified in step 1.  
   - Verify the latest updates: Ensure that the information retrieved is current and reflects any recent changes to the parental leave policy.  

3. Generate answer  
   - Compile the information into a coherent format: Summarize the parental


# Agent: Query Refinement Specialist
## Task: Refine this query: 'what is the parental leave policy?' so it’s optimized for search

In [70]:
from IPython.display import Markdown, display
display(Markdown("### "+ result.raw))

### The parental leave policy for AKAIWorks LLP outlines the procedures and entitlements for time off work for all employees, which includes full-time, part-time, temporary, and contract employees. This policy is designed to support employees in taking necessary leave for personal, medical, or family reasons while ensuring that organizational operations run smoothly. The policy emphasizes fair and consistent administration of leave and compliance with relevant legal regulations.

1. **Eligibility Criteria**: All employees are eligible to apply for parental leave as per the guidelines set forth in the policy, encompassing full-time, part-time, and temporary staff.

2. **Duration**: While specific details regarding the duration of parental leave were not retrieved from the data,

---